In [1]:
import os
import mne

sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = os.path.join(sample_data_folder, 'MEG', 'sample', 'sample_audvis_filt-0-40_raw.fif')
raw = mne.io.read_raw_fif(sample_data_raw_file, verbose=False)

events_file = os.path.join(sample_data_folder, 'MEG', 'sample', 'sample_audvis_filt-0-40_raw-eve.fif')
events = mne.read_events(events_file)

%matplotlib widget

In [2]:
fig = raw.plot()
fig.canvas.key_press_event('a')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Generating annotations programatically

Use the lower-level artifact detection function to get an events array telling us where the blinks are, then automatically add “bad_blink” annotations around them (this is not necessary when using `create_eog_epochs()`, it is done here just to show how annotations are added non-interactively). We’ll start the annotations 250 ms before the blink and end them 250 ms after it:

In [3]:
eog_events = mne.preprocessing.find_eog_events(raw)
onsets = eog_events[:, 0] / raw.info['sfreq'] - 0.25
durations = [0.5] * len(eog_events)
descriptions = ['bad blink'] * len(eog_events)
blink_annot = mne.Annotations(onsets, durations, descriptions, orig_time=raw.info['meas_date'])

raw.set_annotations(blink_annot)

EOG channel index for this subject is: [375]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Setting up band-pass filter from 2 - 45 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 2.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 1.75 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 45.25 Hz)
- Filter length: 2048 samples (13.639 sec)

Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition 

<Raw  |  sample_audvis_filt-0-40_raw.fif, n_channels x n_times : 376 x 41700 (277.7 sec), ~3.7 MB, data not loaded>

Looking at the EEG channels (easy to see eye blinks), we can confirm that the annotations are centered around the EOG events

In [4]:
eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)
raw.plot(events=eog_events, order=eeg_picks);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Rejecting Epochs based on channel amplitude

* setting **maximum** acceptable peak-to-peak amplitudes for each channel type in an epoch, using the `reject` parameter. 
* There is also a related parameter, `flat`, that can be used to set **minimum** acceptable peak-to-peak amplitudes for each channel type in an epoch:

In [5]:
reject_criteria = dict(mag=3000e-15, # 3000 fT
                       grad=3000e-13,# 3000 fT/cm
                       eeg=100e-6, # 100 µV
                       eog=200e-6) # 200 µV

flat_criteria = dict(mag=1e-15, # 1 fT
                     grad=1e-13, # 1 fT/cm
                     eeg=1e-6) # 1 µV

The values that are appropriate are dataset- and hardware-dependent, so some trial-and-error may be necessary to find the correct balance between data quality and loss of power due to too many dropped epochs. Here, we’ve set the rejection criteria to be fairly stringent, for illustration purposes.

Two additional parameters, `reject_tmin` and `reject_tmax`, are used to set the temporal window in which to calculate peak-to-peak amplitude for the purposes of epoch rejection. These default to the same `tmin` and `tmax` of the entire epoch. As one example, if you wanted to only apply the rejection thresholds to the portion of the epoch that occurs before the event marker around which the epoch is created, you could set `reject_tmax=0`.

In [6]:
epochs = mne.Epochs(raw,
                    events,
                    tmin=-0.2,
                    tmax=0.5,
                    reject_tmax=0,
                    reject=reject_criteria,
                    flat=flat_criteria,
                    reject_by_annotation=False,
                    preload=True)

epochs.plot_drop_log();

319 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
Created an SSP operator (subspace dimension = 4)
4 projection items activated
Loading data for 319 events and 106 original time points ...
    Rejecting  epoch based on EEG : ['EEG 001', 'EEG 002', 'EEG 003', 'EEG 007']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EEG : ['EEG 001', 'EEG 003', 'EEG 007']
    Rejecting  epoch based on MAG : ['MEG 1711']
    Rejecting  epoch based on EEG : ['EEG 007']
    Rejecting  epoch based on EEG : ['EEG 003']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EEG : ['EEG 001', 'EEG 002', 'EEG 003']
8 bad epochs dropped


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Notice that we’ve passed `reject_by_annotation=False` above, in order to isolate the effects of the rejection thresholds. If we re-run the epoching with `reject_by_annotation=True` (the default) we see that the rejections due to EEG and EOG channels have disappeared (suggesting that those channel fluctuations were probably blink-related, and were subsumed by rejections based on the “bad blink” label).

In [7]:
epochs = mne.Epochs(raw,
                    events,
                    tmin=-0.2,
                    tmax=0.5,
                    reject_tmax=0,
                    reject=reject_criteria,
                    flat=flat_criteria,
                    preload=True)

epochs.plot_drop_log();

319 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
Created an SSP operator (subspace dimension = 4)
4 projection items activated
Loading data for 319 events and 106 original time points ...
    Rejecting  epoch based on MAG : ['MEG 1711']
66 bad epochs dropped


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

More importantly, note that many more epochs are rejected (~20% instead of ~2.5%) when rejecting based on the blink labels, underscoring why it is usually desirable to repair artifacts rather than exclude them.

In [8]:
print(epochs.drop_log)

[['bad blink'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['bad blink'], ['bad blink'], ['bad blink'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['bad blink'], ['bad blink'], [], ['bad blink'], ['bad blink'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['bad blink'], ['bad blink'], ['bad blink'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['bad blink'], ['bad blink'], ['bad blink'], ['bad blink'], ['bad blink'], ['bad blink'], ['bad blink'], [], ['bad blink'], ['bad blink'], [], [], [], [], [], [], [], [], [], [], [], [], ['bad blink'], ['bad blink'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['MEG 1711'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['bad blink'], ['bad blink'], ['bad blink'], ['bad blink'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['bad blink'], ['bad blink'], [], [], [

In [9]:
epochs.drop_bad()

In [10]:
stronger_reject_criteria = dict(mag=2000e-15, # 2000 fT
                                grad=2000e-13, # 2000 fT/cm
                                eeg=100e-6, # 100 µV
                                eog=100e-6) # 100 µV

epochs.drop_bad(reject=stronger_reject_criteria)
print(epochs.drop_log)

    Rejecting  epoch based on MAG : ['MEG 0111', 'MEG 0141', 'MEG 1411', 'MEG 1421', 'MEG 1431', 'MEG 1541']
    Rejecting  epoch based on MAG : ['MEG 0111', 'MEG 0141', 'MEG 1421', 'MEG 1431', 'MEG 1541', 'MEG 2621']
    Rejecting  epoch based on MAG : ['MEG 0111', 'MEG 0141', 'MEG 1411', 'MEG 1421', 'MEG 1431', 'MEG 1441', 'MEG 1541', 'MEG 2621']
    Rejecting  epoch based on MAG : ['MEG 0111', 'MEG 0141', 'MEG 1421', 'MEG 1431', 'MEG 1541', 'MEG 2621']
    Rejecting  epoch based on MAG : ['MEG 0141', 'MEG 1421', 'MEG 1431', 'MEG 1541', 'MEG 2621']
    Rejecting  epoch based on MAG : ['MEG 0111', 'MEG 0141', 'MEG 1421', 'MEG 1431']
    Rejecting  epoch based on MAG : ['MEG 0111', 'MEG 0141', 'MEG 1411', 'MEG 1421', 'MEG 1431', 'MEG 1541', 'MEG 2621']
    Rejecting  epoch based on MAG : ['MEG 0111', 'MEG 0141', 'MEG 1421', 'MEG 1431', 'MEG 1541', 'MEG 2621']
    Rejecting  epoch based on MAG : ['MEG 1421', 'MEG 1431', 'MEG 2621']
    Rejecting  epoch based on MAG : ['MEG 0111', 'MEG 0